# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import dice_ml
from dice_ml import Dice
from dice_ml.utils import helpers  # helper functions

In [2]:
%load_ext autoreload
%autoreload 2

## Preliminaries: Loading the data and ML model

In [3]:
dataset = helpers.load_adult_income_dataset().sample(5000)  # downsampling to reduce ML model fitting time
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [4]:
target = dataset["income"]

# Split data into train and test
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

In [5]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [6]:
exp = Dice(d, m, method="random")
query_instance = x_train[1:2]
e1 = exp.generate_counterfactuals(query_instance, total_CFs=10, desired_range=None,
                                  desired_class="opposite",
                                  permitted_range=None, features_to_vary="all")
e1.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.96it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,23,Private,Some-college,Single,Professional,White,Male,10,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,60,-,Doctorate,-,-,-,-,74,1
1,89,-,Doctorate,-,-,-,-,-,1
2,66,-,Prof-school,-,-,-,-,-,-
3,-,-,Doctorate,Married,-,-,-,60,1
4,50,-,Bachelors,Married,-,-,-,-,1
5,38,-,Prof-school,-,-,-,-,-,-
6,56,-,-,-,-,-,-,38,1
7,56,-,-,-,-,-,-,36,1
8,-,Self-Employed,Prof-school,Widowed,-,-,-,-,1
9,58,-,Prof-school,-,-,-,-,-,1


These can now be used to calculate the feature importance scores. 

In [7]:
imp = exp.local_feature_importance(query_instance, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

[{'education': 0.8, 'age': 0.8, 'hours_per_week': 0.4, 'marital_status': 0.3, 'workclass': 0.1, 'occupation': 0.0, 'race': 0.0, 'gender': 0.0}]


Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [8]:
imp = exp.local_feature_importance(query_instance, posthoc_sparsity_param=None)
print(imp.local_importance)

100%|██████████| 1/1 [00:00<00:00,  4.97it/s]

[{'age': 0.9, 'education': 0.5, 'hours_per_week': 0.4, 'marital_status': 0.3, 'workclass': 0.1, 'gender': 0.1, 'occupation': 0.0, 'race': 0.0}]


## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [9]:
cobj = exp.global_feature_importance(x_train[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

100%|██████████| 10/10 [00:01<00:00,  5.93it/s]

{'education': 0.55, 'hours_per_week': 0.46, 'age': 0.41, 'marital_status': 0.36, 'occupation': 0.22, 'gender': 0.13, 'workclass': 0.08, 'race': 0.03}


## Convert the counterfactual output to json

In [10]:
json_str = cobj.to_json()
print(json_str)

{"test_data": [[[40, "Private", "Prof-school", "Single", "Professional", "White", "Female", 50, 1]], [[23, "Private", "Some-college", "Single", "Professional", "White", "Male", 10, 0]], [[33, "Private", "Assoc", "Married", "Sales", "White", "Male", 36, 0]], [[43, "Private", "HS-grad", "Married", "Blue-Collar", "White", "Male", 42, 1]], [[37, "Private", "HS-grad", "Divorced", "Sales", "White", "Female", 60, 0]], [[30, "Private", "HS-grad", "Married", "Blue-Collar", "White", "Male", 65, 1]], [[55, "Self-Employed", "Bachelors", "Married", "White-Collar", "White", "Male", 40, 1]], [[48, "Private", "School", "Divorced", "Blue-Collar", "White", "Female", 32, 0]], [[54, "Private", "HS-grad", "Married", "Blue-Collar", "White", "Male", 48, 1]], [[19, "Other/Unknown", "School", "Single", "Other/Unknown", "White", "Female", 35, 0]]], "cfs_list": [[[40, "Private", "HS-grad", "Single", "Service", "White", "Female", 50, 0], [40, "Private", "Assoc", "Single", "White-Collar", "White", "Female", 50, 0]

## Convert the json output to a counterfactual object

In [11]:
imp_r = imp.from_json(json_str)
print([o.visualize_as_dataframe(show_only_changes=True) for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)

Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,40,Private,Prof-school,Single,Professional,White,Female,50,1



Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,HS-grad,-,Service,-,-,-,0
1,-,-,Assoc,-,White-Collar,-,-,-,0
2,-,Other/Unknown,HS-grad,-,-,-,-,-,0
3,-,-,Some-college,-,Sales,-,-,-,0
4,-,Self-Employed,School,-,-,-,-,-,0
5,-,-,-,-,-,-,Male,18,0
6,-,-,-,Separated,Blue-Collar,-,-,-,0
7,-,-,Assoc,Married,-,-,-,-,0
8,-,-,Doctorate,-,-,-,-,36,0
9,-,-,Assoc,-,-,-,-,-,0


Query instance (original outcome : 0)


...

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,19,Other/Unknown,School,Single,Other/Unknown,White,Female,35,0



Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,54,Self-Employed,Some-college,Married,Sales,-,-,-,1
1,72,-,Prof-school,Divorced,-,-,-,86,1
2,78,-,Prof-school,-,White-Collar,-,-,67,1
3,74,-,Prof-school,-,Sales,-,-,59,1
4,-,-,Doctorate,Married,Professional,-,Male,86,1
5,78,-,Prof-school,-,-,-,-,67,1
6,78,-,Doctorate,Married,-,-,Male,95,1
7,78,-,Doctorate,Married,-,-,Male,-,1
8,73,-,Prof-school,Married,-,Other,-,63,1
9,-,-,Doctorate,Married,Professional,-,Male,-,1


[None, None, None, None, None, None, None, None, None, None]
[{'education': 0.8, 'occupation': 0.4, 'workclass': 0.2, 'marital_status': 0.2, 'hours_per_week': 0.2, 'gender': 0.1, 'age': 0.0, 'race': 0.0}, {'age': 0.9, 'hours_per_week': 0.8, 'education': 0.5, 'marital_status': 0.4, 'workclass': 0.0, 'occupation': 0.0, 'race': 0.0, 'gender': 0.0}, {'education': 0.6, 'age': 0.4, 'hours_per_week': 0.4, 'workclass': 0.2, 'occupation': 0.2, 'gender': 0.1, 'marital_status': 0.0, 'race': 0.0}, {'marital_status': 0.5, 'hours_per_week': 0.5, 'age': 0.3, 'education': 0.3, 'gender': 0.2, 'workclass': 0.1, 'occupation': 0.1, 'race': 0.0}, {'education': 0.7, 'marital_status': 0.7, 'age': 0.2, 'hours_per_week': 0.2, 'occupation': 0.1, 'gender': 0.1, 'workclass': 0.0, 'race': 0.0}, {'age': 0.6, 'hours_per_week': 0.4, 'gender': 0.2, 'workclass': 0.1, 'education': 0.1, 'marital_status': 0.1, 'occupation': 0.1, 'race': 0.0}, {'occupation': 0.5, 'education': 0.4, 'marital_status': 0.4, 'age': 0.3, 'hours_